In [ ]:
import sys 
from pathlib import Path

main_path = Path('..').resolve()
sys.path.append(str(main_path))
# sys.path.append(str(main_path / 'fge') )

from fge import ModelBuilder, Dataset, TreeBuilder, FeatureInteractionTree
import itertools
import os
import shap
import time
import pickle

In [ ]:
def cal_time(start: float, end: float):
    time_cost = end - start
    mins = int(time_cost//60)
    secs = time_cost - mins*60
    return mins, secs

def get_shap_interaction_values(explainer, dataset, group_id: None | int=None):
    start = time.time()
    print('Getting Interaction Values via SHAP package, might take a while...')
    if group_id is None:
        # run all data
        print(f'Processing: # of data = {len(dataset.data["X_train"])}, # of features = {len(dataset.feature_names)}')
        shap_interactions = explainer.shap_interaction_values(dataset.data['X_train'])
    else:
        # run seperate group
        # polifitter should mimic subset of group data to build tree
        data = dataset[group_id]

        print(f'Processing: # of data = {len(data["X_train"])}, # of features = {len(dataset.feature_names)}')
        shap_interactions = explainer.shap_interaction_values(data['X_train'])
    end = time.time()
    mins, secs = cal_time(start, end)
    print(f'Cost time: {mins:d} mins {secs:.2f} secs')
    return shap_interactions, end-start

In [ ]:
seed = 7
dataset_names = ['titanic', 'adult', 'boston', 'california']
force_rerun=False

data_folder = Path('../data').resolve()
cache_folder = Path('../checkpoints').resolve()
model_folder = cache_folder / 'models'
if not model_folder.exists():
    model_folder.mkdir(parents=True)

model_kwargs = {
    'titanic': dict(eta=0.15, max_depth=6, subsample=1.0, seed=seed, num_rounds=500),
    'adult': dict(eta=0.1, max_depth=6, subsample=1.0, seed=seed, num_rounds=500),
    'california': dict(eta=0.1, max_depth=6, subsample=1.0, seed=seed, num_rounds=500),
    'boston': dict(eta=0.1, max_depth=6, subsample=1.0, seed=seed, num_rounds=500),
    'ames': dict(eta=0.1, max_depth=6, subsample=1.0, seed=seed, num_rounds=500),
}

In [ ]:
for ds in dataset_names:
    print(f'---- Running Dataset: {ds} ----')
    system = '_win' if os.name == 'nt' else ''
    filename = f'{ds}{system}.pickle'
    if (not force_rerun) and (cache_folder / filename).exists():
        print(f'Pass {ds}, since the file exists')
    else:        
        dataset = Dataset(dataset_name=ds, data_folder=data_folder, seed=seed)
        model_builder = ModelBuilder()
        results = model_builder.train(dataset, **model_kwargs[ds])
        
        explainer = shap.TreeExplainer(results['model'])

        # for record
        results['dataset'] = dataset
        # results['explainer'] = explainer
        results['kwargs'] = model_kwargs[ds]
        results['bst_num_rounds'] = model_builder.best_num_rounds 
        siv, siv_time_cost = get_shap_interaction_values(explainer, dataset, group_id=None)
        results['siv'] = siv
        results['siv_time_cost'] = siv_time_cost

        print(f'test score: {results["score"]}')
        with (model_folder / filename).open('wb') as file:
            pickle.dump(results, file)

In [ ]:
from collections import defaultdict
from tqdm import tqdm
import numpy as np
import pandas as pd
from fge.functions import *
from fge import PolyFitter
from anytree import Node

import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Any

from copy import deepcopy
from fge.utils import flatten
import xgboost as xgb

In [ ]:
def predict_save_data(dataset, model, typ='train'):
    df = dataset.data[f'X_{typ}'].copy()
    data = xgb.DMatrix(df)
    pred = model.predict(data)
    df['target'] = dataset.data[f'y_{typ}']
    df['pred'] = pred
    df.to_csv(f'../cache/preprocessed/{ds}_{typ}.csv', index=False)

dataset_names = ['titanic', 'adult', 'boston', 'california']
for ds in dataset_names:
    system = '_win' if os.name == 'nt' else ''
    filename = f'{ds}{system}.pickle'
    with (model_folder / filename).open('rb') as file:
        res = pickle.load(file)
    dataset = res['dataset']
    model = res['model']
    print(dataset.feature_names)
    print(ds, dataset.data['X_test'].shape)
    
    predict_save_data(dataset, model, typ='train')
    predict_save_data(dataset, model, typ='test')

In [ ]:
# dataset_names = ['titanic', 'adult', 'boston', 'california']
ds = 'california'
system = '_win' if os.name == 'nt' else ''
filename = f'{ds}{system}.pickle'
with (model_folder / filename).open('rb') as file:
    res = pickle.load(file)
print(res.keys())
dataset = res['dataset']
siv = res['siv']
score = res['score']

In [ ]:
ds = 'adult'
system = '_win' if os.name == 'nt' else ''
filename = f'{ds}{system}.pickle'
with (model_folder / filename).open('rb') as file:
    res = pickle.load(file)
print(res.keys())
dataset = res['dataset']
siv = res['siv']
score = res['score']

In [ ]:
d = dataset.loader()

In [ ]:
# init for trees
score_methods = {
    'base': g_base,
    'abs': g_abs,
    'abs_interaction': g_abs_only_interaction,
    'ratio': g_ratio,
}
polyfitter = PolyFitter(dataset.task_type, dataset.data, original_score=score)

In [ ]:
score_method = 'ratio' # should we consider main effect? seperately? or together?

feature_names = np.arange(siv.shape[-1])
g_fn = score_methods[score_method]
siv_scores = g_fn(siv)
n_features = siv_scores.shape[-1]

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
names = dataset.data['X_train'].columns
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

for ax, (g_name, g_function) in zip(axes.flatten(), score_methods.items()):
    s = g_function(siv)
    sns.heatmap(pd.DataFrame(s, index=names, columns=names), annot=True, fmt='.4f', cmap='coolwarm', ax=ax)
    # ax.matshow(, )
    ax.set_title(f'function: {g_name}')
    # for (i, j), z in np.ndenumerate(s):
    #     ax.text(j, i, '{:0.4f}'.format(z), ha='center', va='center', fontsize=8)
plt.tight_layout()
plt.show()

In [ ]:
import networkx as nx
import pygraphviz
from io import BytesIO
from PIL import Image as PILImage

In [ ]:
r_l, c_l = np.tril_indices(siv_scores.shape[1], -1)
# to check the threshold
coor_scores = siv_scores[r_l, c_l]
qs = np.percentile(coor_scores, [25, 50, 75])
mean, std = coor_scores.mean(), coor_scores.std()
print(f'mean={mean:.4f}, std={std:.4f}, 25%={qs[0]:.4f}, 50%={qs[1]:.4f}, 75%={qs[2]:.4f}')

s = siv_scores.copy()
condition = s < qs[2]
print(f'no satisfied ratio: {condition.sum()} / {np.prod(s.shape)}')
# set condition
s[condition] = 0.0

In [ ]:
G = pygraphviz.AGraph(directed=False)
G.layout(prog='dot')
G = nx.Graph()
for i, j in zip(r_l, c_l):
    if s[i, j] > 0:
        G.add_node(i)
        G.add_node(j)
        G.add_edge(i, j, weight=f'{s[i, j]:.4f}')

In [ ]:
imgbuf = BytesIO()
G.draw(imgbuf, format='png', prog='dot')
img = PILImage.open(imgbuf)
img

In [ ]:
G = nx.Graph()
for i, j in zip(r_l, c_l):
    if s[i, j] > 0:
        G.add_node(i)
        G.add_node(j)
        G.add_edge(i, j, weight=f'{s[i, j]:.4f}')
pos = nx.spring_layout(G)
nx.draw_networkx(G, pos, with_labels=True)
labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
plt.show()

In [ ]:
# arguments
score_method = 'ratio' # should we consider main effect? seperately? or together?
n_select_ratio = 0.5
n_filter_ratio = 0.1
n_search = 5  # beam search
max_iter = None
select_method = 'random'  # random / sort
filter_method = 'random'  # random / sort
rt_only_best = True
verbose = False

In [ ]:
feature_names = np.arange(siv.shape[-1])
g_fn = score_methods[score_method]
siv_scores = g_fn(siv)

n_feature = siv_scores.shape[-1]

if max_iter is None:
    max_iter = n_feature
    non_root_tree = False
else:
    max_iter = min(max_iter, n_feature)
    non_root_tree = True if max_iter < n_feature else False

In [ ]:
# initialize: sum of effects
k = 0  # iteration
sum_effects = siv_scores.sum(0)
# gaps = {'origin': polyfitter.original_score}
nodes = {}
for i, name in enumerate(feature_names):
    n = Node(
        name=str(name), 
        parent=None, 
        score=sum_effects[i], 
        interaction=0.0, 
        k=k, 
        gap=None, 
        summary=None
    )
    nodes[i] = n

prev_step = [nodes]

In [ ]:
def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum()

def get_scores(siv_scores, nodes_to_run):
    scores = {}
    for cs in itertools.combinations(nodes_to_run, 2):
        if cs not in scores.keys():
            r, c = list(zip(*itertools.product(flatten(cs), flatten(cs))))
            scores[cs] = siv_scores[r, c].sum()
    return scores

def select_nodes(nodes_to_run, n_select, method='sort'):
    if method == 'random':
        filtered_nodes_to_run = _random_selection(container=nodes_to_run, n=n_select)
    elif method == 'sort':
        # probs = softmax(np.array(list(map(lambda x: x[1], nodes_to_run))))
        # idxes = np.random.choice(np.arange(len(probs)), size=(n_select,), replace=False, p=probs)
        sorted_nodes_to_run = sorted(nodes_to_run, key=lambda x: x[1], reverse=True)
        filtered_nodes_to_run = list(map(lambda x: x[0], sorted_nodes_to_run[:n_select]))
    return filtered_nodes_to_run

def filter_scores(scores, n_filter, method='sort'):
    if len(scores) == 1:
        return list(scores.keys())

    if method == 'random':
        filtered_keys = _random_selection(container=list(scores.items()), n=n_filter)
    elif method == 'sort':
        filtered = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        filtered_keys = list(map(lambda x: x[0], filtered[:n_filter]))
    return filtered_keys

def _random_selection(container: List[Tuple[Any, float]], n: int):
    if len(container) <= n:
        return list(map(lambda x: x[0], container))
    else:
        # probs = softmax(np.array(list(map(lambda x: x[1], container))))
        # idxes = np.random.choice(np.arange(len(probs)), size=(n,), replace=False, p=probs)
        idxes = np.random.choice(np.arange(len(container)), size=(n,), replace=False)
        return [container[i][0] for i in idxes]

In [ ]:
# Step in iteration
k += 1

In [ ]:
h_k = 0

hypothesis = []

while prev_step:
    h_k += 1
    print(f'--- Hypothesis {h_k} ---')
    prev_nodes = prev_step.pop(0)

    # Phase: Nodes Selection
    print('- Phase: Nodes Selection')
    ## sort method
    nodes_to_run = [(key, n.score) for key, n in prev_nodes.items()]
    n_select = max(2, round(len(nodes_to_run)*n_select_ratio))
    filtered_nodes_to_run = select_nodes(nodes_to_run=nodes_to_run, n_select=n_select, method=select_method)
    print(f'Nodes to run: {filtered_nodes_to_run}')

    # Phase: Calculate Scores
    print('- Phase: Calculate Scores')
    scores = get_scores(siv_scores, nodes_to_run=filtered_nodes_to_run)
    print(f'Scores: {scores}')

    # Phase: Filter Scores
    print('- Phase: Filter Scores')
    n_combinations = len(scores)
    n_filter = max(1, round(n_combinations*n_filter_ratio))
    filtered_keys = filter_scores(scores, n_filter, method=filter_method)
    print(filtered_keys)

    # Phase: Calculate Gap
    for cmbs in filtered_keys:
        combined_keys = list(filter(lambda x: isinstance(x, tuple), prev_nodes.keys()))
        combined_keys_history = set()
        list(flatten(combined_keys, res=combined_keys_history))
        trials = list(combined_keys_history) + [cmbs]
        print(f'Trials: {trials}')
        gap, model = polyfitter.get_interaction_gap(trials)
        hypothesis.append((cmbs, gap, deepcopy(prev_nodes), model))

    print()

sorted_hypothesis = sorted(hypothesis, key=lambda x: x[1])[:n_search]
print('top_hypothesis')
print(list(map(lambda x: x[:2], sorted_hypothesis)))

In [ ]:
def get_value_and_interaction(siv_scores, cmbs):
    r_l, c_l = np.tril_indices(siv_scores.shape[1], -1)
    cmbs_flattend = list(flatten(cmbs))
    cmbs_idx = np.arange(len(r_l))[np.isin(r_l, cmbs_flattend) & np.isin(c_l, cmbs_flattend)]

    r, c = list(zip(*itertools.product(flatten(cmbs), flatten(cmbs))))
    value = siv_scores[r, c].sum()
    interaction = siv_scores[r_l, c_l][cmbs_idx].sum()
    return value, interaction

prev_step = []
for i, (cmbs, gap, new_nodes, model) in enumerate(sorted_hypothesis):
    value, interaction = get_value_and_interaction(siv_scores, cmbs)
    feature_name = '+'.join([str(feature_names[i]) for i in flatten(cmbs)])     
    children = [new_nodes[c] for c in cmbs]
    new_nodes[cmbs] = Node(
        name=feature_name, 
        score=value, 
        interaction=interaction, 
        children=children, 
        k=k,
        gap=gap,
        model=model
    )
    # add impossibles cmbs
    for c in cmbs:
        new_nodes.pop(c)
    prev_step.append(new_nodes)

In [ ]:
for i, nodes in enumerate(prev_step):
    print(f'Hypothesis {i}')
    x = dict(filter(lambda x: isinstance(x[0], tuple), nodes.items()))
    print(x.keys())
    print()

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.compose import make_column_selector as selector
from sklearn.pipeline import make_pipeline

In [ ]:
X_train = dataset.data['X_train']
y_train = dataset.data['y_train']
X_test = dataset.data['X_test']
y_test = dataset.data['y_test']

In [ ]:
model = make_pipeline(StandardScaler(), polyfitter.task_model(**polyfitter.args))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
polyfitter.task_metric(y_test, y_pred)

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

def nested_map(x, feature_names):
    if isinstance(x, int):
        return feature_names[x]

    if isinstance(x, tuple):
        return tuple(nested_map(ele, feature_names) for ele in x)

    if isinstance(x, list):
        res = []
        for ele in x:
            res.append(nested_map(ele, feature_names))
        return res

class InteractionTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, trials, feature_names):
        super().__init__()
        self.trials = trials
        features = {}
        for i, name in enumerate(feature_names):
            features[i] = name

        for i, name in zip(trials, nested_map(trials, feature_names)):
            features[i] = name

        self.feature_names = features

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        if isinstance(X, pd.DataFrame):
            X = X.values
        for cmbs in self.trials:
            X = np.concatenate([X, X[:, list(flatten(cmbs))].prod(1, keepdims=True)], axis=1)
        return X

    def get_feature_names(self):
        return self.feature_names

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
print(scaler.feature_names_in_)
X_t = scaler.transform(X_train)
X_t.shape

In [ ]:
trials = [(5, (9, 2)), (9, 2), (3, 4), (11, 8)]
# Feature = namedtuple('Feature', ['name'])
feature_names = X_train.columns
features = {}
for i, name in enumerate(feature_names):
    features[i] = name

for i, name in zip(trials, nested_map(trials, feature_names)):
    features[i] = name
features

In [ ]:
for cmbs in trials:
    X_t = np.concatenate([X_t, X_t[:, list(flatten(cmbs))].prod(1, keepdims=True)], axis=1)
X_t.shape

In [ ]:
for cmbs in trials:
    X_t = np.concatenate([X_t, X_t[:, list(flatten(cmbs))].prod(1, keepdims=True)], axis=1)
X_t.shape

In [ ]:
p = make_pipeline(StandardScaler(), InteractionTransformer(trials=trials, feature_names=list(X_train.columns)), polyfitter.task_model(**polyfitter.args))
p.fit(X_train, y_train)

In [ ]:
p[-1].coef_.round(4), p[-1].intercept_

In [ ]:
p[1]

In [ ]:
preprocessor, _ = polyfitter.get_preprocessor(X_train)
preprocessor

In [ ]:
preprocessor.fit(X_train)

In [ ]:
model = make_pipeline(StandardScaler(), PolynomialFeatures(degree=2, interaction_only=True, include_bias=False), polyfitter.task_model(**polyfitter.args))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
polyfitter.task_metric(y_test, y_pred)

In [ ]:
model[1].get_feature_names_out(model[0].feature_names_in_)

In [ ]:
polyfitter.task_model(polyfitter.args)

In [ ]:
processed = preprocessor.fit_transform(X_train)

In [ ]:
processed.toarray().shape

In [ ]:
PolynomialFeatures(degree=2, interaction_only=True, include_bias=True)

In [ ]:
from fge import TreeBuilder

In [ ]:
tree_builder = TreeBuilder(dataset, original_score=score)

In [ ]:
score_method = 'ratio' # should we consider main effect? seperately? or together?
n_select_ratio = 0.1
n_filter_ratio = 0.1
n_search = 5  # beam search
max_iter = None
select_method = 'sort'  # random / sort
filter_method = 'sort'  # random / sort 
rt_only_best = True
verbose = False

trees = tree_builder.build(
    score_method=score_method, 
    siv=siv, 
    n_select_ratio=n_select_ratio,
    n_filter_ratio=n_filter_ratio, 
    n_search=n_search,
    max_iter=None,
    select_method=select_method,
    filter_method=filter_method,
    rt_only_best=False,
    verbose=1
)

In [ ]:
tree_builder.n_filter

In [ ]:
trees

In [ ]:
r = trees[0]

In [ ]:
for tree in trees:
    roots.append(list(map(lambda x: x[1], filter(lambda x: isinstance(x[0], tuple), tree.items()))))

In [ ]:
t = [FeatureInteractionTree(r) for r in roots[0]]

In [ ]:
t